### This notebook presents the fine-tuning workflow applied to the Mistral-7B-Instruct model using instruction-style formatting, with a focus on aligning responses to emotionally grounded conversational data.

### Dataset used: Daily Dialog Dataset

### Importing Necessary Libraries

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
import unsloth
import torch
import numpy as np
import transformers
import bitsandbytes as bnb
import xformers
import accelerate
import peft
import datasets
import trl

## Model Prep

In [ ]:
from unsloth import FastLanguageModel
import os
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

FastLanguageModel.forbid_torch_xformers = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name =  "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.18: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.18 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Model Prep

In [ ]:
import torch
from unsloth import FastLanguageModel
import os
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

FastLanguageModel.forbid_torch_xformers = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name =  "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.18: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.18 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Prep

In [ ]:
from datasets import load_dataset, Dataset
import re

def split_into_instruction_parts(formatted_str):
    try:
        human_text, bot_text = formatted_str.split("<bot>:")
        human_text = human_text.replace("<human>:", "").strip()

        # Extract instruction and input from 'text:' if present
        if "text:" in human_text:
            instruction, input_part = human_text.split("text:", 1)
        else:
            instruction = human_text
            input_part = ""

        return {
            "instruction": instruction.strip(),
            "input": input_part.strip(),
            "output": bot_text.strip(),
            "text": f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
\n### Instruction:\n{instruction.strip()}\n\n### Input:\n{input_part.strip()}\n\n### Response:\n{bot_text.strip()} </s>"""
        }
    except Exception as e:
        return {
            "instruction": "",
            "input": "",
            "output": "",
            "text": f"[ERROR] Failed to parse: {formatted_str}\nError: {e}"
        }

dataset_emotions = load_dataset("sebdg/go_emotions")
updated_dataset = dataset_emotions.map(split_into_instruction_parts, input_columns=["formatted"])
updated_dataset = updated_dataset.remove_columns(['labels', 'label_text', 'formatted'])

# Verify updated columns
print(updated_dataset.column_names)

Map:   0%|          | 0/168971 [00:00<?, ? examples/s]

Map:   0%|          | 0/42243 [00:00<?, ? examples/s]

{'train': ['text', 'instruction', 'input', 'output'], 'test': ['text', 'instruction', 'input', 'output']}


In [ ]:
print(updated_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'instruction', 'input', 'output'],
        num_rows: 168971
    })
    test: Dataset({
        features: ['text', 'instruction', 'input', 'output'],
        num_rows: 42243
    })
})


In [ ]:
print(updated_dataset['train'][0]["text"])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert in sentiment analysis and emotions detection, provide a list of emotions expressed in the following

### Input:
Big if true

### Response:
This text contains the folliwing emotions: neutral. </s>


### Saving the dataset in hugging face

In [ ]:
from datasets import Dataset
from huggingface_hub import HfApi

dataset_name = "sourname/dailydialog-instruct-dataset"
updated_dataset.push_to_hub(dataset_name)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/169 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/43 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/635 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sourname/dailydialog-instruct-dataset/commit/7afa12c9119fbfb39309627bf6bdb5de16b56bf9', commit_message='Upload dataset', commit_description='', oid='7afa12c9119fbfb39309627bf6bdb5de16b56bf9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/sourname/dailydialog-instruct-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='sourname/dailydialog-instruct-dataset'), pr_revision=None, pr_num=None)

#### Loading dataset from huggingface

In [ ]:
from datasets import load_dataset, Dataset
dataset_emotions = load_dataset("sourname/dailydialog-instruct-dataset")

README.md:   0%|          | 0.00/491 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/24.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/6.00M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/168971 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/42243 [00:00<?, ? examples/s]

In [ ]:
print(dataset_emotions)

DatasetDict({
    train: Dataset({
        features: ['text', 'instruction', 'input', 'output'],
        num_rows: 168971
    })
    test: Dataset({
        features: ['text', 'instruction', 'input', 'output'],
        num_rows: 42243
    })
})


In [ ]:
print(dataset_emotions['test'][12]["text"])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert in sentiment analysis and emotions detection, provide a list of emotions expressed in the following

### Input:
Its like the worlds most infuriating reality show.

### Response:
This text contains the folliwing emotions: anger. </s>


## Train the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_emotions['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/168971 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.043 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 168,971 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: virigineniaishwarya2 (virigineniaishwarya2-university-at-buffalo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,3.667900
2,3.520000
3,2.945700
4,2.443700
5,1.862100
6,1.257000
7,1.127700
8,0.902100
9,0.960200
10,0.887600


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

292.2421 seconds used for training.
4.87 minutes used for training.
Peak reserved memory = 7.359 GB.
Peak reserved memory for training = 0.316 GB.
Peak reserved memory % of max memory = 49.922 %.
Peak reserved memory for training % of max memory = 2.144 %.


## Inference

In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{} </s>"""

### Sample 1

In [ ]:
sample = dataset_emotions['test'][13]
formatted = alpaca_prompt.format(sample['instruction'], sample['input'], "")

inputs = tokenizer(
[
    formatted,

], return_tensors = "pt").to("cuda")
print("\nGenerated: ")
model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)# Your template
print("\nReference: ", sample['output'])


Generated: 
<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert in sentiment analysis and emotions detection, provide a list of emotions expressed in the following

### Input:
Wow, way to be gorgeous as yourself!

### Response:
 </s> This text contains the folliwing emotions: admiration. </s>

Reference:  This text contains the folliwing emotions: admiration.


### Sample 1

In [ ]:
sample = dataset_emotions['test'][3457]
formatted = alpaca_prompt.format(sample['instruction'], sample['input'], "")

inputs = tokenizer(
[
    formatted,

], return_tensors = "pt").to("cuda")
print("\nGenerated: ")
model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)# Your template
print("\nReference: ", sample['output'])


Generated: 
<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert in sentiment analysis and emotions detection, provide a list of emotions expressed in the following

### Input:
I would gladly commit insurance fraud to save my child.

### Response:
 </s> This text contains the folliwing emotions: neutral. </s>

Reference:  This text contains the folliwing emotions: neutral.


###Sample 2

In [ ]:
sample = dataset_emotions['test'][241]
formatted = alpaca_prompt.format(sample['instruction'], sample['input'], "")

inputs = tokenizer(
[
    formatted,

], return_tensors = "pt").to("cuda")
print("\nGenerated: ")
model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)# Your template
print("\nReference: ", sample['output'])


Generated: 
<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert in sentiment analysis and emotions detection, provide a list of emotions expressed in the following

### Input:
Thanks. I was diagnosed with BP 1 after the hospitalization as well.

### Response:
 </s> This text contains the folliwing emotions: gratitude. </s>

Reference:  This text contains the folliwing emotions: gratitude.


###Sample 3

In [ ]:
sample = dataset_emotions['test'][12]
formatted = alpaca_prompt.format(sample['instruction'], sample['input'], "")

inputs = tokenizer(
[
    formatted,

], return_tensors = "pt").to("cuda")
print("\nGenerated: ")
model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)# Your template
print("\nReference: ", sample['output'])


Generated: 
<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert in sentiment analysis and emotions detection, provide a list of emotions expressed in the following

### Input:
Its like the worlds most infuriating reality show.

### Response:
 </s> This text contains the folliwing emotions: annoyance. </s>

Reference:  This text contains the folliwing emotions: anger.


###Sample 5

In [ ]:
sample = dataset_emotions['test'][9852]
formatted = alpaca_prompt.format(sample['instruction'], sample['input'], "")

inputs = tokenizer(
[
    formatted,

], return_tensors = "pt").to("cuda")
print("\nGenerated: ")
model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)# Your template
print("\nReference: ", sample['output'])


Generated: 
<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert in sentiment analysis and emotions detection, provide a list of emotions expressed in the following

### Input:
Id buy those shoes and walk in the dirtiest places just to piss [NAME] off.

### Response:
 </s> This text contains the folliwing emotions: anger. </s>

Reference:  This text contains the folliwing emotions: anger, annoyance.


## Evaluation

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm
import evaluate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
"""

# Initialize outputs
predictions, references, results = [], [], []

# Use a manageable subset (can adjust)
test_dataset = dataset_emotions['test'].select(range(5000))

# Disable tokenizer padding warnings
torch.set_grad_enabled(False)

# Main evaluation loop
for idx, sample in enumerate(tqdm(test_dataset, desc="Evaluating")):
    prompt = alpaca_prompt.format(sample["instruction"], sample["input"])

    # Tokenize (batch of 1) and move to device
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

    # Generate prediction
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    output_ids = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=False,
        # num_beams=4,
        early_stopping=True
    )

    # Decode and extract only response
    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract just the emotion label
    pred_emotion = (decoded.split("### Response:")[-1].strip().replace("</s>", "")).split(":")[-1].strip().rstrip(".")
    ref_emotion = (sample["output"].strip().replace("</s>", "")).split(":")[-1].strip().rstrip(".")

    # Collect outputs
    predictions.append(pred_emotion)
    references.append(ref_emotion)
    results.append({
        "input": sample["input"],
        "reference": ref_emotion,
        "predicted": pred_emotion
    })

In [ ]:
import pandas as pd
df_results = pd.DataFrame(results)

df_results.head(20)

,input,reference,predicted
0,Sorry ideologically yeah. I meant morally.,embarrassment,neutral
1,Man we are sloppy,neutral,neutral
2,Geez. Why is Duchene their 3C?,curiosity,neutral
3,"I kind of hate it when I agree with you, but d...",approval,neutral
4,Ugh I'm sorry man! Jerk off until your heart i...,remorse,neutral
5,As the Dark Lord [NAME] commands.,neutral,neutral
6,He’s gonna have a great season,"admiration, approval",neutral
7,Or just head on to Galleon's Grave. I just hat...,"annoyance, disapproval",neutral
8,What type of migraine do you think it is? Hope...,"caring, curiosity",neutral
9,Probably from all the teens that get drunk and...,neutral,neutral


In [ ]:
reference_sentences = list(df_results['reference'])
generated_sentences = list(df_results['predicted'])

### Metric Evaluation

In [ ]:
!pip install bleu rouge bert_score evaluate

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
import evaluate

# Init rouge
rouge = evaluate.load("rouge")

# Initialize metric lists
bleu_1 = []
bleu_2 = []
bleu_3 = []
bleu_4 = []
meteor_scores = []
rouge1 = []
rouge2 = []
rougeL = []

# Loop over each row
for i, row in df_results.iterrows():
    reference = row["reference"]
    predicted = row["predicted"]

    # BLEU
    smoothie = SmoothingFunction().method4
    bleu_1.append(
        sentence_bleu([reference.strip().split()], predicted.strip().split(), weights=(1, 0, 0, 0), smoothing_function=smoothie)
    )
    bleu_2.append(
        sentence_bleu([reference.strip().split()], predicted.strip().split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
    )
    bleu_3.append(
        sentence_bleu([reference.strip().split()], predicted.strip().split(), weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
    )
    bleu_4.append(
        sentence_bleu([reference.strip().split()], predicted.strip().split(), weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
    )

    # METEOR (safe)
    try:
        m_score = meteor_score([reference.strip().split()], predicted.strip().split())
        meteor_scores.append(m_score)
    except Exception as e:
        print(f"METEOR error at row {i}: {e}")
        continue

    # ROUGE
    rouge_result = rouge.compute(predictions=[predicted], references=[reference])
    rouge1.append(rouge_result["rouge1"])
    rouge2.append(rouge_result["rouge2"])
    rougeL.append(rouge_result["rougeL"])

In [3]:
print("\n=== Average BLEU Scores ===")
print(f"BLEU-1: {sum(bleu_1)/len(bleu_1):.4f}")
print(f"BLEU-2: {sum(bleu_2)/len(bleu_2):.4f}")
print(f"BLEU-3: {sum(bleu_3)/len(bleu_3):.4f}")
print(f"BLEU-4: {sum(bleu_4)/len(bleu_4):.4f}")

if meteor_scores:
    print("\n=== Average METEOR ===")
    print(f"METEOR: {sum(meteor_scores)/len(meteor_scores):.4f}")
else:
    print("\n=== Average METEOR ===")
    print("No METEOR scores computed.")

print("\n=== ROUGE Scores ===")
print(f"ROUGE-1: {sum(rouge1)/len(rouge1):.4f}")
print(f"ROUGE-2: {sum(rouge2)/len(rouge2):.4f}")
print(f"ROUGE-L: {sum(rougeL)/len(rougeL):.4f}")


=== Average BLEU Scores ===
BLEU-1: 0.2901
BLEU-2: 0.1897
BLEU-3: 0.1023
BLEU-4: 0.0076

=== Average METEOR ===
METEOR: 0.1676

=== ROUGE Scores ===
ROUGE-1: 0.3433
ROUGE-2: 0.0000
ROUGE-L: 0.3433


### BERT

In [1]:
import bert_score

# Extract predicted and reference lists
predictions = df_results["predicted"].tolist()
references = df_results["reference"].tolist()

# Run BERTScore (default model: roberta-large)
P, R, F1 = bert_score.score(predictions, references, lang="en", verbose=True)

# Print average scores
print("\n=== BERTScore ===")
print(f"Precision: {P.mean().item():.4f}")
print(f"Recall:    {R.mean().item():.4f}")
print(f"F1:        {F1.mean().item():.4f}")


=== BERTScore ===
Precision: 0.8473
Recall:    0.8189
F1:        0.8302
